In [1]:
from array import array
import subprocess
import math
import ROOT
import os
import numpy as np

Welcome to JupyROOT 6.29/01


In [2]:
buildcuda = '/home/mdessole/Projects/ROOT/genvectorx/build_cuda/'

In [3]:
buildoneapi = '/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/'

In [4]:
buildopensycl = '/home/mdessole/Projects/ROOT/genvectorx/build_opensycl/'

In [5]:
!ls /home/mdessole/Projects/ROOT/genvectorx/build_opensycl/testx

CMakeFiles	     InvariantMasses	  SInvariantMasses
cmake_install.cmake  InvariantMassesSYCL  SInvariantMassesSYCL


In [6]:
!ls /home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx

CMakeFiles	     InvariantMasses	  SInvariantMasses
cmake_install.cmake  InvariantMassesSYCL  SInvariantMassesSYCL


In [7]:
!echo $LD_LIBRARY_PATH

/opt/intel/oneapi/tbb/2021.10.0/env/../lib/intel64/gcc4.8:/opt/intel/oneapi/debugger/2023.2.0/gdb/intel64/lib:/opt/intel/oneapi/debugger/2023.2.0/libipt/intel64/lib:/opt/intel/oneapi/debugger/2023.2.0/dep/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/lib/x64:/opt/intel/oneapi/compiler/2023.2.1/linux/lib/oclfpga/host/linux64/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/compiler/lib/intel64_lin:/home/mdessole/Projects/sycl-blas/sycl-blas_build/benchmark/googlebench-build/src:/opt/OpenBLAS/lib:/home/mdessole/Projects/sycl-blas/sycl-blas_build/lib:/home/mdessole/Projects/sycl-blas/sycl-blas/lib:/home/mdessole/Projects/OpenSYCL/OpenSYCL_install/lib:/usr/local/cuda-11.8/lib64:/home/mdessole/Projects/ROOT/root/lib


In [8]:
sizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [9]:
Ssizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000,200000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [10]:
def result2list(exe, N, nruns):
    print(exe, str(int(N)), str(int(nruns)))
    result = subprocess.run([exe, str(int(N)), str(int(nruns)) ], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(result)
    if len(result)<=0:
        return []
    result = result.replace("\r","").replace("\n","")
    time = np.zeros(nruns)
    for r in range(nruns):
        i = result.find("time")
        j = result.find("(s)")
        if (i<0) or (j<0):
            break
        time[r] = float(result[i+5:j])
        result = result[j+3:]
    #endif
    return time

In [11]:
def run_benchmark(builddir, sizes, nruns = 1, testname = "InvariantMasses", btype = 'cpu'):
    timeopt = array( 'd' )
    timestd = array( 'd' )
    
    if btype == 'cpu':
        exe = os.path.join(builddir,'testx',testname)
    elif btype == 'cuda':
        exe = os.path.join(builddir,'testx',testname+'CUDA')
    elif btype == 'sycl':
        exe = os.path.join(builddir,'testx',testname+'SYCL')
    #endif
    
    if (not os.path.exists(exe)):
        print("Executable", exe, "not found!")
        return timeopt, timestd
    
    for i, N in enumerate(sizes):
        time = result2list(exe,N,nruns)
        if (len(time)<=0):
            return timeopt, timestd
        print(N, time, '\n')
        timeopt.append(np.amin(time))
        timestd.append(np.std(time)) 
    #endfor
    return timeopt, timestd

In [13]:
timecpu, stdcpu = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'cpu')

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/InvariantMasses 1000 3
RVec cpu time 6.1e-05 (s)
RVec cpu time 6.1e-05 (s)
RVec cpu time 6.1e-05 (s)

1000.0 [6.1e-05 6.1e-05 6.1e-05] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/InvariantMasses 10000 3
RVec cpu time 0.000623 (s)
RVec cpu time 0.000607 (s)
RVec cpu time 0.000606 (s)

10000.0 [0.000623 0.000607 0.000606] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/InvariantMasses 100000 3
RVec cpu time 0.006213 (s)
RVec cpu time 0.006297 (s)
RVec cpu time 0.006233 (s)

100000.0 [0.006213 0.006297 0.006233] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/InvariantMasses 1000000 3
RVec cpu time 0.065139 (s)
RVec cpu time 0.061928 (s)
RVec cpu time 0.062407 (s)

1000000.0 [0.065139 0.061928 0.062407] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/InvariantMasses 5000000 3
RVec cpu time 0.331508 (s)
RVec cpu time 0.331815 (s)
RVec cpu time 0.328972 (s)

5000000.0 [0.331508

In [14]:
timecpu

array('d', [6.1e-05, 0.000606, 0.006213, 0.061928, 0.328972, 0.651606, 3.29044, 6.50448])

In [15]:
Stimecpu, Sstdcpu = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'cpu')

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/SInvariantMasses 1000 3
SINGLE_PRECISION defined 
RVec cpu time 4.6e-05 (s)
RVec cpu time 4.6e-05 (s)
RVec cpu time 4.6e-05 (s)

1000.0 [4.6e-05 4.6e-05 4.6e-05] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/SInvariantMasses 10000 3
SINGLE_PRECISION defined 
RVec cpu time 0.000449 (s)
RVec cpu time 0.000449 (s)
RVec cpu time 0.000449 (s)

10000.0 [0.000449 0.000449 0.000449] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/SInvariantMasses 100000 3
SINGLE_PRECISION defined 
RVec cpu time 0.00454 (s)
RVec cpu time 0.00446 (s)
RVec cpu time 0.004467 (s)

100000.0 [0.00454  0.00446  0.004467] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/SInvariantMasses 1000000 3
SINGLE_PRECISION defined 
RVec cpu time 0.046633 (s)
RVec cpu time 0.044825 (s)
RVec cpu time 0.045019 (s)

1000000.0 [0.046633 0.044825 0.045019] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/SInvariantMasses 5000

In [16]:
Stimecpu

array('d', [4.6e-05, 0.000449, 0.00446, 0.044825, 0.234479, 0.476881, 2.37433, 4.72168, 9.39256])

In [43]:
timeoneapi, stdoneapi = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/InvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000695 (s)
sycl time 0.000347 (s)
sycl time 0.00029 (s)

1000.0 [0.000695 0.000347 0.00029 ] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/InvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001076 (s)
sycl time 0.000513 (s)
sycl time 0.000322 (s)

10000.0 [0.001076 0.000513 0.000322] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/InvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.006066 (s)
sycl time 0.004856 (s)
sycl time 0.00347 (s)

100000.0 [0.006066 0.004856 0.00347 ] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/InvariantMassesSYCL 10

In [44]:
timeoneapi

array('d', [0.00029, 0.000322, 0.00347, 0.027566, 0.245257, 0.473385, 2.2328, 4.35295])

In [45]:
Stimeoneapi, Sstdoneapi = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/SInvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000485 (s)
sycl time 0.000317 (s)
sycl time 0.00021 (s)

1000.0 [0.000485 0.000317 0.00021 ] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/SInvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000552 (s)
sycl time 0.000368 (s)
sycl time 0.000268 (s)

10000.0 [0.000552 0.000368 0.000268] 

/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx/SInvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001318 (s)
sycl time 0.001116 (s)
sycl time 0.000941 (s)

100000.0 [0.001318 0.001116 0.000941] 


In [26]:
timeopensycl, stdopensycl = run_benchmark(buildopensycl, sizes[:-1],  nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorx/build_opensycl/testx/InvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.
[hipSYCL Warning] OpenCL device NVIDIA GeForce RTX 3060 does not have a valid USM provider. Memory allocations are not possible on that device.
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.00104 (s)
sycl time 0.000383 (s)
sycl time 0.00037 (s)

1000.0 [0.00104  0.000383 0.00037 ] 

/home/mdessole/Projects/ROOT/genvectorx/build_opensycl/testx/InvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.
[hipSYCL Warning] OpenCL device 

In [46]:
Stimeoneapi

array('d', [0.00021, 0.000268, 0.000941, 0.006832, 0.030812, 0.057504, 0.274305, 0.530976, 1.05419])

In [27]:
timeopensycl, sizes[:]

(array('d', [0.00037, 0.000429, 0.00222, 0.016401, 0.080974, 0.155269, 0.734168]),
 array('d', [1000.0, 10000.0, 100000.0, 1000000.0, 5000000.0, 10000000.0, 50000000.0, 100000000.0]))

In [28]:
Stimeopensycl, Sstdopensycl = run_benchmark(buildopensycl, Ssizes[:-1], testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorx/build_opensycl/testx/SInvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.
[hipSYCL Warning] OpenCL device NVIDIA GeForce RTX 3060 does not have a valid USM provider. Memory allocations are not possible on that device.
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001013 (s)
sycl time 0.000204 (s)
sycl time 0.000298 (s)

1000.0 [0.001013 0.000204 0.000298] 

/home/mdessole/Projects/ROOT/genvectorx/build_opensycl/testx/SInvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info querie

In [29]:
timecuda, stdcuda = run_benchmark(buildcuda, sizes, nruns = 3, btype = 'cuda')

/home/mdessole/Projects/ROOT/genvectorx/build_cuda/testx/InvariantMassesCUDA 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000318 (s)
cuda time 4.5e-05 (s)
cuda time 4.3e-05 (s)

1000.0 [3.18e-04 4.50e-05 4.30e-05] 

/home/mdessole/Projects/ROOT/genvectorx/build_cuda/testx/InvariantMassesCUDA 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000418 (s)
cuda time 0.000192 (s)
cuda time 0.000172 (s)

10000.0 [0.000418 0.000192 0.000172] 

/home/mdessole/Projects/ROOT/genvectorx/build_cuda/testx/InvariantMassesCUDA 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.001669 (s)
cuda time 0.001415 (s)
cuda time 0.001312 (s)

100000.0 [0.001669 0.001415 0.001312] 

/home/mdessole/Projects/ROOT/genvectorx/build_cuda/testx/InvariantMassesCUDA 1000000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.01257 (s)
cuda ti

In [30]:
timecuda

array('d', [4.3e-05, 0.000172, 0.001312, 0.011896, 0.060011, 0.119447, 0.553895, 1.09766])

In [31]:
Stimecuda, Sstdcuda = run_benchmark(buildcuda, Ssizes, nruns = 3, testname = "SInvariantMasses", btype = 'cuda')

/home/mdessole/Projects/ROOT/genvectorx/build_cuda/testx/SInvariantMassesCUDA 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000288 (s)
cuda time 2.9e-05 (s)
cuda time 2.8e-05 (s)

1000.0 [2.88e-04 2.90e-05 2.80e-05] 

/home/mdessole/Projects/ROOT/genvectorx/build_cuda/testx/SInvariantMassesCUDA 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000311 (s)
cuda time 8.6e-05 (s)
cuda time 8.9e-05 (s)

10000.0 [3.11e-04 8.60e-05 8.90e-05] 

/home/mdessole/Projects/ROOT/genvectorx/build_cuda/testx/SInvariantMassesCUDA 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000848 (s)
cuda time 0.000646 (s)
cuda time 0.000581 (s)

100000.0 [0.000848 0.000646 0.000581] 

/home/mdessole/Projects/ROOT/genvectorx/build_cuda/testx/SInvariantMassesCUDA 1000000 3
ROOT_MEAS_TIMING defined 

In [47]:
n = len(sizes) 
speedupopensycl = array('d', [timecpu[i]/timeopensycl[i] for i in range(n-1)])
speeduponeapi   = array('d', [timecpu[i]/timeoneapi[i] for i in range(n)])
speedupcuda     = array('d', [timecpu[i]/timecuda[i] for i in range(n)])

In [48]:
n = len(Ssizes)
Sspeedupopensycl = array('d', [Stimecpu[i]/Stimeopensycl[i] for i in range(n-1)])
Sspeeduponeapi   = array('d', [Stimecpu[i]/Stimeoneapi[i] for i in range(n)])
Sspeedupcuda     = array('d', [Stimecpu[i]/Stimecuda[i] for i in range(n)])

In [37]:
print(Sspeedupopensycl)
print(Sspeeduponeapi)
print(Sspeedupcuda)

array('d', [0.22549019607843138, 1.6943396226415095, 3.7542087542087543, 6.073848238482384, 7.5487412272229735, 7.933208011711472, 8.327680603830762, 8.367871010503968])
array('d', [0.21800947867298578, 1.7269230769230772, 4.764957264957266, 6.655530809205642, 7.7278689605167745, 7.963678567850105, 8.764728899651526, 8.797893728700183, 8.879838145479987])
array('d', [1.6428571428571428, 5.22093023255814, 7.676419965576592, 9.215666118421051, 10.059159159159158, 10.211147274206672, 10.264044093807414, 10.568575048235543, 10.768172500607621])


In [31]:
val = list(sizes)
sizes_gb = [v*8*4/1e9 for v in val] #(nb of doubles)*(bytes for doubles)*(4=dim of LVector)/(bytes in GB)
sizes_gb = array('d', sizes_gb)
sizes_gb

array('d', [3.2e-05, 0.00032, 0.0032, 0.032, 0.16, 0.32, 1.6, 3.2])

In [37]:
val = list(Ssizes)
Ssizes_gb = [v*4*4/1e9 for v in val] #(nb of floats)*(bytes for floats)*(4=dim of LVector)/(bytes in GB)
Ssizes_gb = array('d', Ssizes_gb)
Ssizes_gb

array('d', [1.6e-05, 0.00016, 0.0016, 0.016, 0.08, 0.16, 0.8, 1.6, 3.2])

In [49]:
p = ROOT.TGraph(len(sizes)-1, sizes[:-1], speedupopensycl)
g = ROOT.TGraph(len(sizes), sizes, speeduponeapi)
q = ROOT.TGraph(len(sizes), sizes, speedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec")
q.GetXaxis().SetTitle( 'Input size (# doubles)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("speedup.png")

Info in <TCanvas::Print>: png file speedup.png has been created


In [53]:
len(Ssizes), Ssizes, len(Sspeedupopensycl)

(9,
 array('d', [1000.0, 10000.0, 100000.0, 1000000.0, 5000000.0, 10000000.0, 50000000.0, 100000000.0, 200000000.0]),
 8)

In [50]:
p = ROOT.TGraph(len(Ssizes)-1, Ssizes[:-1], Sspeedupopensycl)
g = ROOT.TGraph(len(Ssizes), Ssizes, Sspeeduponeapi)
q = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec - Single Precision")
q.GetXaxis().SetTitle( 'Input size (# floats)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sspeedup.png")

Info in <TCanvas::Print>: png file Sspeedup.png has been created


In [41]:
p = ROOT.TGraphErrors(len(sizes)-1, sizes[:-1], timeopensycl, stdopensycl)
g = ROOT.TGraphErrors(len(sizes), sizes, timeoneapi, stdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, timecuda, stdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, timecpu, stdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times")
r.GetXaxis().SetTitle( 'Input size (# doubles)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("exectimes.png")

Info in <TCanvas::Print>: png file exectimes.png has been created


In [42]:
p = ROOT.TGraphErrors(len(sizes)-1, sizes[:-1], Stimeopensycl, Sstdopensycl)
g = ROOT.TGraphErrors(len(sizes), sizes, Stimeoneapi, Sstdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, Stimecuda, Sstdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, Stimecpu, Sstdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times - Single Precision")
r.GetXaxis().SetTitle( 'Input size (# floats)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sexectimes.png")

Info in <TCanvas::Print>: png file Sexectimes.png has been created
